In [39]:
# Libraries and a constant

import numpy as np
from random import choice
from math import sqrt, log

num_simulations = 1000

In [40]:
class TicTacToeState:
    """ A state of the game TicTacToe
    """
    def __init__(self):
        self.board = [[0, 0, 0] for _ in range(3)]
        self.next_to_move = 1

    def is_game_over(self):
        # Horizontal and Vertical checks
        for i in range(3):
            if self.board[i][0] == self.board[i][1] == self.board[i][2] != 0:
                return True
            if self.board[0][i] == self.board[1][i] == self.board[2][i] != 0:
                return True
        # Diagonal checks
        if self.board[0][0] == self.board[1][1] == self.board[2][2] != 0:
            return True
        if self.board[2][0] == self.board[1][1] == self.board[0][2] != 0:
            return True
        # Draw check
        if all(cell != 0 for row in self.board for cell in row):
            return True
        return False

    def get_legal_actions(self):
        actions = []
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:
                    actions.append((i, j))
        return actions

    def take_action(self, action):
        new_state = TicTacToeState()
        new_state.board = [row.copy() for row in self.board]
        new_state.board[action[0]][action[1]] = self.next_to_move
        new_state.next_to_move = 3 - self.next_to_move
        return new_state

    def get_reward(self):
        for i in range(3):
            if self.board[i][0] == self.board[i][1] == self.board[i][2] != 0:
                return 1 if self.board[i][0] == 1 else -1
            if self.board[0][i] == self.board[1][i] == self.board[2][i] != 0:
                return 1 if self.board[0][i] == 1 else -1
        if self.board[0][0] == self.board[1][1] == self.board[2][2] != 0:
            return 1 if self.board[0][0] == 1 else -1
        if self.board[2][0] == self.board[1][1] == self.board[0][2] != 0:
            return 1 if self.board[2][0] == 1 else -1
        return 0

    def __str__(self):
        return '\n'.join(' '.join(str(cell) for cell in row) for row in self.board)


In [41]:
class MCTSNode:
    # Initialize MCTSNode object
    def __init__(self, game_state, parent=None, action=None, prior=0):
        self.game_state = game_state
        self.parent = parent
        self.action = action
        self.children = []
        self.visits = 0
        self.value = 0
        self.prior = prior
        self.total_reward = 0

    def add_child(self, child_node):
        # Add child node to MCTSNode
        self.children.append(child_node)

    def update(self, reward):
        self.visits += 1
        self.value += reward
        self.total_reward += reward  
    
    def q(self):
        # return average reward
        if self.visits == 0:
            return 0
        else:
            return self.total_reward / self.visits

    def get_value(self):
        # return total value
        return self.value * self.visits
    
    def is_fully_expanded(self):
        # Check if MCTSNode is fully expanded
        return len(self.children) == len(self.game_state.get_legal_actions())
      
    def is_terminal(self):
        return self.game_state.is_game_over()

    def select_child(self, c_param=sqrt(2)):
        choices_weights = [
            ((child.value / (child.visits + 1e-8) + c_param * child.prior * np.sqrt(self.visits) / (1 + child.visits)))
            for child in self.children
        ]
        return self.children[np.argmax(choices_weights)]

    def is_leaf(self):
        return len(self.children) == 0

    def expand(self):
        for action in self.game_state.get_legal_actions():
            if not any(child.action == action for child in self.children):
                child_game_state = self.game_state.take_action(action)
                child_node = MCTSNode(child_game_state, parent=self, action=action)
                self.add_child(child_node)
                return child_node
        raise Exception("Should never reach here. If it does, there's a bug in the code.")


In [48]:
class TicTacToeModel:
    def __init__(self, num_simulations=1000):
        self.num_simulations = num_simulations

    def learn(self, num_games):
        for i in range(num_games):
            print(f"Starting game {i+1}")
            self.play_game()

    def play_game(self):
        state = TicTacToeState()
        while not state.is_game_over():
            print(state)  # Print current game state
            if state.next_to_move == 1:
                action = self.uct_search(state)
            else:
                action = choice(state.get_legal_actions())
            print(f"Action taken: {action}")  # Print action taken
            state = state.take_action(action)

    def uct_search(self, game_state):
        root = MCTSNode(game_state)
        for _ in range(self.num_simulations):
            child = tree_policy(root)
            reward = default_policy(child.game_state)
            backup(child, reward)
        return best_child(root).action



In [49]:



def tree_policy(node):
    """ Selection and expansion
    """
    while not node.is_terminal():
        if not node.is_fully_expanded():
            return node.expand()
        else:
            node = best_child(node)
    return node

def default_policy(state):
    """ Simulation
    """
    while not state.is_game_over():
        state = state.take_action(choice(state.get_legal_actions()))
    return state.get_reward()

def backup(node, reward):
    """ Backpropagation
    """
    while node is not None:
        node.visits += 1
        node.total_reward += reward
        node = node.parent

def best_child(node, c_param=sqrt(2)):
    """ Best child selection
    """
    choices_weights = [
        (c.visits, c.q() + c_param * np.sqrt(node.visits) / (1 + c.visits)) for c in node.children
    ]
    return node.children[choices_weights.index(max(choices_weights))]

Starting game 1
0 0 0
0 0 0
0 0 0
Action taken: (0, 1)
0 1 0
0 0 0
0 0 0
Action taken: (1, 0)
0 1 0
2 0 0
0 0 0
Action taken: (0, 0)
1 1 0
2 0 0
0 0 0
Action taken: (1, 2)
1 1 0
2 0 2
0 0 0
Action taken: (0, 2)
Starting game 2
0 0 0
0 0 0
0 0 0
Action taken: (0, 0)
1 0 0
0 0 0
0 0 0
Action taken: (2, 0)
1 0 0
0 0 0
2 0 0
Action taken: (0, 1)
1 1 0
0 0 0
2 0 0
Action taken: (1, 0)
1 1 0
2 0 0
2 0 0
Action taken: (0, 2)
Starting game 3
0 0 0
0 0 0
0 0 0
Action taken: (0, 0)
1 0 0
0 0 0
0 0 0
Action taken: (1, 0)
1 0 0
2 0 0
0 0 0
Action taken: (0, 1)
1 1 0
2 0 0
0 0 0
Action taken: (2, 2)
1 1 0
2 0 0
0 0 2
Action taken: (0, 2)
Starting game 4
0 0 0
0 0 0
0 0 0
Action taken: (0, 1)
0 1 0
0 0 0
0 0 0
Action taken: (2, 0)
0 1 0
0 0 0
2 0 0
Action taken: (0, 0)
1 1 0
0 0 0
2 0 0
Action taken: (1, 1)
1 1 0
0 2 0
2 0 0
Action taken: (0, 2)
Starting game 5
0 0 0
0 0 0
0 0 0
Action taken: (0, 0)
1 0 0
0 0 0
0 0 0
Action taken: (0, 2)
1 0 2
0 0 0
0 0 0
Action taken: (1, 1)
1 0 2
0 1 0
0 0 0
Actio